In [3]:
import os
import sys

sys.path.append("../code")
from util import extract_pdf_to_txt

In [5]:
done = os.listdir("../text")
for filename in os.listdir("../pdf"):
    if (filename + ".txt") in done:
        continue
    extract_pdf_to_txt(filename, filename, "local")

In [7]:
done = os.listdir("../text")
len(done)

116

# Google Sheets


In [82]:
import gspread

In [8]:
# connect to the service account
gc = gspread.service_account(filename="../creds.json")

sh = gc.open("Paper Extraction Evaluation").sheet1
sh.update_cell(10, 3, "helloooo")

FALSE
None


{'spreadsheetId': '1KunlZNe9InDjmYCOpgj4wgIAScObBxh237bCOpUcUCQ',
 'updatedRange': 'Sheet1!C10',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

# RegEx


In [135]:
import re

import sys

sys.path.append("../code")

from util import extract_doi_to_txt, get_string_from_text_file

In [213]:
# ***Last line should not contain "|"***
regex_method = r"""
    ^\s*\d?[.]?\s?(Our(\s))?Method(s)?(ology)?\s*$|# "method" "methods" "methodology"
    ^\s*\d?[.]?\s?Material(s)?((\s)and(\s)Methods)?\s*$| # "materials and methods" "material" "materials"
    ^\s*\d?[.]?\s?Approach(es)?\s*$ # "approach"
"""

# ***Last line should not contain "|"***
regex_result_only = r"""
    ^\s*\d?[.]?\s?Result(s)?((\s)and(\s)(discussion|analysis))?\s*$| # "result" "results" "results and discussion"
    ^\s*\d?[.]?\s?Empirical(\s)Result(s)?\s*$ # "result" "results" "results and discussion"
    
"""
# ***Last line should not contain "|"***
regex_result_others = r"""
    ^\s*\d?[.]?\s?Experiment(s)?((\s)and(\s)discussion(s)?)?\s*$| # "experiment" "experiments" "experiments and discussions"
    ^\s*\d?[.]?\s?Evaluation(s)?((\s)and(\s)comparison(s)?)?\s*$| # "evaluation" "evaluations" "evaluation and comparisons"
    ^\s*\d?[.]?\s?Comparison(s)?((\s)with(\s)State-of-the-arts)?\s*$ # "comparison" "comparison with state-of-the-arts"
"""

# ***Last line should not contain "|"***
regex_extra = r"""
    ^\s*\d?[.]?\s?(Further(\s))?Discussion(s)?\s*$| # "discussion" "discussions"
    ^\s*\d?[.]?\s?Reference(s)?\s*$| # "reference" "references"
    ^\s*\d?[.]?\s?Conclusion\s*$| # "conclusion"
    ^\s*\d?[.]?\s?Literature(\s)Review\s*$| # "literature review"
    ^\s*\d?[.]?\s?Introduction\s*$| # "introduction"
    ^\s*\d?[.]?\s?Abstract\s*$| # "abstract"
    ^\s*\d?[.]?\s?Limitation(s)?(\s)and(\s)Discussion(s)?\s*$| # "limitation" "limitations"
    ^\s*\d?[.]?\s?Appendi(x)?(ces)?\s*$| # "appendix" "appendices"
    ^\s*\d?[.]?\s?Bibliography\s*$| # "bibliography"
    ^\s*\d?[.]?\s?Acknowledgement(s)?\s*$| # "acknowledgement" "acknowledgements"
    ^\s*\d?[.]?\s?Tables(\s)and(\s)Figures\s*$| # "tables and figures"
    ^\s*\d?[.]?\s?(Related|Previous)(\s)Work\s*$| # "related work"
    ^\s*\d?[.]?\s?Analysis\s*$| # "analysis"
    ^\s*\d?[.]?\s?(Additional(\s))?Ablation(\s)Stud(y|ies)\s*$| # "analysis" "ablation study" "additional ablation study"
    ^\s*\d?[.]?\s?Preliminar(y|ies)\s*$| # "preliminary"
    ^\s*\d?[.]?\s?Experiment(\s)Setup\s*$ # "experiment setup"
"""

regex_total = (
    regex_method
    + "|"
    + regex_result_only
    + "|"
    + regex_result_others
    + "|"
    + regex_extra
)

# Testing


In [76]:
# custom_regex = "hi"
custom_regex = None
if custom_regex:
    regex_total += "\n    "
    if custom_regex[0] != "|":
        custom_regex = "|" + custom_regex
    regex_total += custom_regex
print(regex_total)

# add custom rules
regex_total_cp = re.compile(regex_total, re.IGNORECASE | re.VERBOSE | re.MULTILINE)
regex_method_cp = re.compile(regex_method, re.IGNORECASE | re.VERBOSE)
regex_result_others_cp = re.compile(regex_result_others, re.IGNORECASE | re.VERBOSE)
regex_result_only_cp = re.compile(regex_result_only, re.IGNORECASE | re.VERBOSE)


    ^\d?[.]?\s?Method(s)?(ology)?$|# "method" "methods" "methodology"
    ^\d?[.]?\s?Material(s)?((\s)and(\s)Methods)?$ # "materials and methods" "material" "materials"
|
    ^\d?[.]?\s?Result(s)?((\s)and(\s)discussion)?$| # "result" "results" "results and discussion"
    ^\d?[.]?\s?Empirical(\s)Result(s)?$ # "result" "results" "results and discussion"
    
|
    ^\d?[.]?\s?Experiment(s)?$| # "experiment" "experiments"
    ^\d?[.]?\s?Evaluation(s)?((\s)and(\s)comparison(s)?)?$ # "evaluation" "evaluations" "evaluation and comparisons"
|
    ^\d?[.]?\s?Discussion(s)?$| # "discussion" "discussions"
    ^\d?[.]?\s?Reference(s)?$| # "reference" "references"
    ^\d?[.]?\s?Conclusion$| # "conclusion"
    ^\d?[.]?\s?Literature(\s)Review$| # "literature review"
    ^\d?[.]?\s?Introduction$| # "introduction"
    ^\d?[.]?\s?Abstract$| # "abstract"
    ^\d?[.]?\s?Limitation(s)?$| # "limitation" "limitations"
    ^\d?[.]?\s?Appendi(x)?(ces)?$| # "appendix" "appendices"
    ^\d?[.]?\s?Bibliography

In [77]:
text = get_string_from_text_file(
    filename="Arjovsky et al. - 2017 - Wasserstein GAN.pdf"
)

method_idx = -1
result_idx = -1
result_flag = False  # because "Results" section can be called many names
section_start_point_list = []
for i, match in enumerate(regex_total_cp.finditer(text)):
    section = match.group()
    if regex_method_cp.search(section):
        method_idx = i

    # If there is an exact match with "Results"
    if regex_result_only_cp.search(section):  # if there is an exact match
        result_idx = i
        result_flag = True
    # If not, and if exact match has not been found yet, look for other names
    if regex_result_others_cp.search(section) and not result_flag:
        result_idx = i

    section_start_point_list.append(match.start())
    print(match)
    print([match.group()])
    # break


print(method_idx)
print(result_idx)
print(section_start_point_list)

reading data from Arjovsky et al. - 2017 - Wasserstein GAN.pdf.txt...
string successfully retrieved.

<re.Match object; span=(140, 154), match='1 Introduction'>
['1 Introduction']
<re.Match object; span=(18649, 18668), match='4 Empirical Results'>
['4 Empirical Results']
<re.Match object; span=(26878, 26892), match='5 Related Work'>
['5 Related Work']
<re.Match object; span=(34696, 34706), match='References'>
['References']
-1
1
[140, 18649, 26878, 34696]


In [101]:
list_length = len(section_start_point_list)
start_idx = section_start_point_list[method_idx]
end_idx = (
    -1 if method_idx >= list_length - 1 else section_start_point_list[method_idx + 1]
)
method_text = text[start_idx:end_idx] if method_idx != -1 else ""
end_idx = None if result_idx >= list_length - 1 else result_idx + 1

start_idx = section_start_point_list[result_idx]
end_idx = (
    -1 if result_idx >= list_length - 1 else section_start_point_list[result_idx + 1]
)
result_text = text[start_idx:end_idx] if result_idx != -1 else ""

# Function Definition


In [167]:
def extract(
    filename: str,
    regex_base: str,
    custom_regex: str | None = None,
    row_idx: int | None = -1,
) -> None:
    print("\n====================================================")
    regex_total = regex_base
    if custom_regex:
        regex_total += "\n    "
        if custom_regex[0] != "|":
            custom_regex = "|" + custom_regex
        regex_total += custom_regex

    # print("Final RegEx:\n", regex_total)

    # add custom rules
    regex_total_cp = re.compile(
        regex_total,
        re.IGNORECASE | re.VERBOSE | re.MULTILINE,
    )
    regex_method_cp = re.compile(regex_method, re.IGNORECASE | re.VERBOSE)
    regex_result_others_cp = re.compile(regex_result_others, re.IGNORECASE | re.VERBOSE)
    regex_result_only_cp = re.compile(regex_result_only, re.IGNORECASE | re.VERBOSE)

    text = get_string_from_text_file(filename)

    method_idx = -1
    result_idx = -1
    result_flag = False  # because "Results" section can be called many names
    section_start_point_list = []
    extracted_section_name_list = []
    for i, match in enumerate(regex_total_cp.finditer(text)):
        section = match.group()
        if regex_method_cp.search(section):
            method_idx = i

        # If there is an exact match with "Results"
        if regex_result_only_cp.search(section):  # if there is an exact match
            result_idx = i
            result_flag = True
        # If not, and if exact match has not been found yet, look for other names
        if regex_result_others_cp.search(section) and not result_flag:
            result_idx = i

        section_start_point_list.append(match.start())
        # print(match)
        extracted_section_name_list.append(match.group().strip())
        print([match.group()])

    list_length = len(section_start_point_list)
    if list_length == 0:
        print("No sections found...")
        return
    start_idx = section_start_point_list[method_idx]
    end_idx = (
        len(text)
        if method_idx >= list_length - 1
        else section_start_point_list[method_idx + 1]
    )
    end_idx = min(end_idx, start_idx + 40000)
    method_text = text[start_idx:end_idx] if method_idx != -1 else ""

    start_idx = section_start_point_list[result_idx]
    end_idx = (
        len(text)
        if result_idx >= list_length - 1
        else section_start_point_list[result_idx + 1]
    )
    # 40000 -> char limit per section (50000 char limit on gs api)
    end_idx = min(end_idx, start_idx + 40000)
    result_text = text[start_idx:end_idx] if result_idx != -1 else ""

    if row_idx != -1:
        # connect to the service account
        gc = gspread.service_account(filename="../creds.json")

        sh = gc.open("Paper Extraction Evaluation").sheet1
        print(f"\nUpdating google sheet row {row_idx}")
        sh.update_cell(row_idx, 1, filename[:-4])
        sh.update_cell(row_idx, 2, "\n".join(extracted_section_name_list))
        sh.update_cell(row_idx, 4, method_text)
        sh.update_cell(row_idx, 7, result_text)
    else:
        print("############ METHOD TEXT:\n", method_text)
        print("############ RESULT TEXT:\n", result_text)
        print(
            "############ Extracted Section List:\n",
            "\n".join(extracted_section_name_list),
        )

# Loop through text files


In [186]:
# row number that has already been executed
breakpoint_row_num = 99
for i, file in enumerate(os.listdir("../text"), 3):
    if i <= breakpoint_row_num:
        continue
    if (i - 3) >= 100:
        break
    extract(file, regex_total, row_idx=i)


reading data from Yang et al. - 2022 - An Empirical Study of GPT-3 for Few-Shot Knowledge.pdf.txt...
string successfully retrieved.

['Abstract']
['Introduction']
['Related Work']
['Conclusion']

Updating google sheet row 100

reading data from Schuhmann et al. - LAION-5B An open large-scale dataset for training.pdf.txt...
string successfully retrieved.

['Abstract']
['1 Introduction']
['2 Related Work']
['8 Conclusion']
['References']

Updating google sheet row 101

reading data from Giarelis et al. - 2023 - Abstractive vs. Extractive Summarization An Exper.pdf.txt...
string successfully retrieved.

['1. Introduction']
['2. Related Work']
['3. Evaluation']
['References']

Updating google sheet row 102


# Individual File


In [214]:
extract(
    "Chen et al. - 2021 - A Training-free and Reference-free Summarization E.pdf",
    regex_total,
    row_idx=15,
)


reading data from Chen et al. - 2021 - A Training-free and Reference-free Summarization E.pdf.txt...
string successfully retrieved.

['Abstract']
['1 Introduction']
['2 Preliminary']
['3 Our Methodology']
['4 Experiment Setup']
['5 Results and Analysis']
['6 Related Work']
['7 Conclusion']
['Acknowledgements']
['References']

Updating google sheet row 15
